In this script, we use the geemap (https://github.com/giswqs/geemap) module. Map is plotted on notebook. A rectangle is drawn onto it, which selects and bounds the intersecting ICESAT2 track within the drawn rectangle. Water occurrence and elevation plots are drawn together for comparison of any relations. Interact is also setup to filter regressions as per the 03_SetupInteract script.
#Plot only withing the bounds of the rectangle for the track-segments within the rectangle bounds

In [1]:
%matplotlib widget

In [2]:
%matplotlib widget

In [3]:
#fork eo_bathymetry; add most necessary scripts; add to repo; push/commit to your fork; create pull request; comes to main repo once gena accepts
#github pull request tutorial; command line or website
#https://guides.github.com/activities/hello-world/

In [4]:
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
import geemap
import ee

In [6]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets



In [7]:
import seaborn as sb

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale
from collections import Counter

In [8]:
df = pd.read_pickle('../data/out/df.pkl')
df2 = pd.read_pickle('../data/out/df2.pkl')
df3 = pd.read_pickle('../data/out/df3.pkl')

In [9]:
Map = geemap.Map()

# Bangladesh
extents = [89.57958670206244, 23.721136875400937, 89.99453684543461, 24.618409448937893] #Extent same as users/gena/AquaMonitor2020/bathymetry-jamuna/bathymetry3-jamuna-2018-2019
zoom = 10

Map = geemap.Map(center=[((23.721136875400937 + 24.618409448937893)/2), ((89.57958670206244 + 89.99453684543461)/2)], zoom = zoom)


#tracks_shp = gpd.read_file(r"../data/icesat-2/icesat2_tracks.shp")
tracks_shp = r"../data/icesat-2/icesat2_tracks.shp"
tracks = geemap.shp_to_ee(tracks_shp)
#Map.addLayer(tracks, {}, 'Tracks')

roi = ee.Geometry.Polygon([[[89.57958670206244, 23.721136875400937],[89.57958670206244, 24.618409448937893],[89.99453684543461, 24.618409448937893],[89.99453684543461, 23.721136875400937]]])
fc = tracks.filterBounds(roi)
clipped = fc.map(lambda f: f.intersection(roi))
Map.addLayer(clipped, {}, 'Tracks')
Map.addLayer(roi,{}, 'Bounding Box')
Map

Map(center=[24.169773162169413, 89.78706177374852], controls=(WidgetControl(options=['position'], widget=HBox(…

<H1> Draw region of interest in the above map </H1>

In [10]:
#widget or inline? Explore further

https://github.com/matplotlib/ipympl <br>
https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html

In [11]:
rect = Map.draw_last_feature

In [71]:
rect

In [72]:
roi2 = ee.FeatureCollection(rect)
selected_track = clipped.filterBounds(roi2)
#Map.addLayer(selected_track,{palette: "red"},'Selected Track')

In [73]:
image = ee.Image().paint(**{
    'featureCollection': selected_track,
    'color': 1,
    'width': 3
})

In [74]:
#clipped = fc.map(lambda f: f.intersection(rect))

#Map.centerObject(ee.FeatureCollection(roi), 12)
Map.addLayer(ee.Image().paint(roi, 0, 2), {'palette': 'yellow'}, 'Rect')
# Map.setCenter(-73.9596, 40.7688, 12)


#Map.addLayer(ee.Image().paint(clipped, 0, 3), {'palette': 'red'}, 'clipped')
Map.addLayer(image, {'palette': 'FF0000'}, 'Selected Track')


In [75]:
Map

Map(bottom=113236.0, center=[24.125448627037315, 89.79743957519533], controls=(WidgetControl(options=['positio…

In [76]:
#selected_track.getInfo()['features']

In [77]:
#type(selected_track.getInfo()['features'])

In [78]:
#selected_track.getInfo()['features']

In [79]:
#selected_track.getInfo()['features'][0]['id']

In [80]:
id = []
for feature in selected_track.getInfo()['features']:
    id.append(feature['properties']['TrackId'])

In [81]:
id

['1239']

In [82]:
#df.track_id == '180'

In [83]:
Map

Map(bottom=113236.0, center=[24.125448627037315, 89.79743957519533], controls=(WidgetControl(options=['positio…

In [85]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))
# (df.series != 'Noise') & (df.series != 'Buffer') & 
#data = df [(df.series != 'Noise') & (df.track_id == '180') & (df.date == '2020-01-07') & (df.beam == 'gt2r')]
data = df [(df.series != 'Noise') & (df.date == '2020-01-07') & (df.beam == 'gt2r')]
#Get bounding coordinates for the rectangle drawn
xlist = []
ylist = []
for x,y in rect.getInfo()['geometry']['coordinates'][0]:
    xlist.append(x)
    ylist.append(y)
xlist.sort()
ylist.sort()

#Filter as per bound coordinates
data = data[data.lon >= xlist[0]]
data = data[(data.lon <= xlist[-1])]
data = data[(data.lat >= ylist[0])]
data = data[(data.lat <= ylist[-1])]

# data = data[data.lon >= ylist[0]]
# data = data[(data.lon <= ylist[-1])]
# data = data[(data.lat >= xlist[0])]
# data = data[(data.lat <= xlist[-1])]

ax[0].plot(data.h, '.', markersize=5, alpha=0.2)
ax[1].plot(data.water_occurrence, '.', markersize=5, alpha=0.2)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
# data = df [(df.series != 'Noise') & (df.track_id == '180') & (df.date == '2020-01-07') & (df.beam == 'gt2r')]
# data

In [ ]:
#twin axis
#check filters

In [48]:
# data = data[data.lon >= xlist[0]]
# data

In [49]:
# xlist[0]

In [50]:
#booleanseries = df.track_id.isin(id)

In [51]:
#booleanseries

In [86]:
start_date = '2018-09-01'
end_date = '2019-09-01'
quantile_cutoff = 0.05

In [87]:
beam_name = df.beam.unique()
series_name = df.series.unique()

def reg(beam_name = 'gt3r', series_name = 'High', start_date = '2018-09-01', end_date = '2019-09-01', quantile_cutoff = 0.05):
    columns = ['track_id','beam', 'series', 'm', 'c', 'R2', 'mse']
    result_df = pd.DataFrame(columns = columns)
    dict_df ={}
    for b in df[df.beam == beam_name].beam.unique():
        new_df = df[df.series != 'Noise']
        new_df.dropna()
        
        #Filter as per bound coordinates
        new_df = new_df[new_df.lon >= xlist[0]]
        new_df = new_df[(new_df.lon <= xlist[-1])]
        new_df = new_df[(new_df.lat >= ylist[0])]
        new_df = new_df[(new_df.lat <= ylist[-1])]
        
        
        for s in new_df[new_df.series == series_name].series.unique():

            #Select only track_idd 622 (Directly passing through the Jamuna river)
            #for track_id in df[df.track_id == '622'].track_id.unique():

            #Run this for a more general analysis    
            for track_id in df[df.track_id.isin(id)].track_id.unique():

                #Select particular beam, series and track_id
                loop_df = df[(df.beam == b) & (df.series == s) & (df.track_id == track_id)]
                if loop_df.empty: continue

                #Filter dates
                loop_df = loop_df[(loop_df['date'] >= start_date) & (loop_df['date'] < end_date)]
                if loop_df.empty: continue

                #Remove water occurance <0
                #loop_df = loop_df[loop_df.water_occurrence >= 0]
                #if loop_df.empty: continue

                #Filter off extreme quantile data
                h_lowlim = loop_df.h.quantile(quantile_cutoff)
                h_uplim = loop_df.h.quantile(1 - quantile_cutoff)
                if(h_uplim < h_lowlim): h_lowlim, h_uplim = h_uplim, h_lowlim
                water_lowlim = loop_df.water_occurrence.quantile(quantile_cutoff)
                water_uplim = loop_df.water_occurrence.quantile(1 - quantile_cutoff)
                if(water_uplim < water_lowlim): water_lowlim, water_uplim = water_uplim, water_lowlim

                loop_df = loop_df[(loop_df['h'] > h_lowlim) & (loop_df['h'] < h_uplim)]
                loop_df = loop_df[(loop_df['water_occurrence'] > water_lowlim) & (loop_df['water_occurrence'] < water_uplim)]
                #Run linear regression model
                linreg = LinearRegression()
                X = loop_df[['h']]
                y = loop_df[['water_occurrence']]
                linreg.fit(X,y)
                y_pred = linreg.predict(X)
                mse = sklearn.metrics.mean_squared_error(y,y_pred)


                dict_df = {'track_id': track_id, 'beam': b, 'series': s, 'm': linreg.coef_, 'c':linreg.intercept_ , 'R2': linreg.score(X,y), 'mse': mse}
                print(dict_df)

                print(b, s)
                %matplotlib inline
                plt.scatter(X,y)
                plt.plot(X,y_pred, color='red')
                plt.show()

                print(linreg.coef_)
                print(linreg.intercept_)
                print(linreg.score(X,y))
                print(mse)
                print("")
                print("")

                result_df = result_df.append(dict_df, ignore_index = True)
                
                
#                 fig, ax = plt.subplots(1, 2, figsize=(20, 5))
#                 # (df.series != 'Noise') & (df.series != 'Buffer') & 
#                 data = df [(df.series != 'Noise') & (df.track_id == '180') & (df.date == '2020-01-07') & (df.beam == 'gt2r')]

#                 ax[0].plot(loop_df.lon, '.', markersize=5, alpha=0.2)
#                 ax[1].plot(X, loop_df, 'k.', markersize=0.1, alpha=0.2)
#                 ax[1].plot(y, loop_df, 'r.', markersize=0.1, alpha=0.2)
#                 plt.show()

        
        
    return 

In [88]:
Map

Map(bottom=113236.0, center=[24.125448627037315, 89.79743957519533], controls=(WidgetControl(options=['positio…

In [89]:
interact(reg, beam_name=beam_name, series_name = series_name, start_date = '2018-09-01', end_date = '2019-09-01', quantile_cutoff = (0.05, 0.35));

interactive(children=(Dropdown(description='beam_name', options=('gt3r', nan, 'gt3l', 'gt2r', 'gt2l', 'gt1r', …